# Teil 2: Training des Modells

Im Unterverzeichnis `04-sheets` sollten nun viele Bilddateien mit 28x28 Pixel großen Ziffern liegen. Wir laden die Bilder, extrahieren das jeweilige Label aus dem Dateinamen und konstruieren ein künstliches neuronales Netzwerk. 60% der Daten nutzen wir für das Training, 20% für die Validierung, und 20% für den Test.

In [ ]:
import tensorflow as tf
import glob
import os
import numpy as np
from PIL import Image

SHEETS_DIR = '04-sheets'
MODEL_DIR = '05-model'

images, labels = [], []
DIGITS = '0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZÄÖÜß'

for path in glob.glob(os.path.join(SHEETS_DIR, "*")):
    img = Image.open(path).convert('L')
    image = np.array(img)
    height, width = image.shape
    for i in range(0, height, 28):
        for j in range(0, width, 28):
            sub = image[i:i+28, j:j+28]
            if not np.all(sub == 255):
                images.append(sub / 255.0)
                labels.append(DIGITS.index(os.path.basename(path).split('.')[0].split('_')[0]))

train_x, train_y = np.array(images), np.array(labels)
data = list(zip(train_x, train_y))
np.random.shuffle(data)

train_data = data[:int(len(data)*0.6)]
valid_data = data[int(len(data)*0.6):int(len(data)*0.8)]
test_data = data[int(len(data)*0.8):]

train_x, train_y = zip(*train_data)
valid_x, valid_y = zip(*valid_data)
test_x, test_y = zip(*test_data)

train_x, train_y = np.array(train_x), np.array(train_y)
valid_x, valid_y = np.array(valid_x), np.array(valid_y)
test_x, test_y = np.array(test_x), np.array(test_y)

# --------------------------------------------
# Alternativ zu deinen eigenen Daten kannst du
# auch den MNIST-Datensatz verwenden.
# Kommentiere dafür diesen Block aus:
# --------------------------------------------
# mnist = tf.keras.datasets.mnist.load_data()
# (train_full_x, train_full_y), (test_x, test_y) = mnist
# train_x, train_y = train_full_x[:-5000], train_full_y[:-5000]
# valid_x, valid_y = train_full_x[-5000:], train_full_y[-5000:]
# train_x, valid_x, test_x = 1.0 - train_x / 255.0, 1.0 - valid_x / 255.0, 1.0 - test_x / 255.0

print(f"✅ Trainingsdaten: {train_y.shape[0]}, Validierungsdaten: {valid_y.shape[0]}, Testdaten: {test_y.shape[0]}")

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(40, activation='softmax'),
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

## Trainingsphase

Als nächstes wird das Netzwerk trainiert und das resultierende Modell gespeichert.

In [ ]:
model.fit(train_x, train_y, epochs=30, validation_data=(valid_x, valid_y))
model.save("my_keras_model.h5")
print("✅ Modell trainiert und gespeichert")

## Qualität des Modells

Wir lassen uns die Qualität des Modell ausgeben, indem wir Loss und Accuracy des Test-Sets bestimmen:

In [ ]:
loss, accuracy = model.evaluate(test_x, test_y, verbose=0)
print("Das trainierte Modell erreicht einen Loss von {:.4f} und eine Accuracy von {:.2f}% auf dem Testdatensatz.".format(loss, accuracy * 100))

Wir probieren nun anhand einiger zufällig gewählter Ziffern aus, was das Modell erkennt:

In [ ]:
import matplotlib.pyplot as plt

def plot_images_with_labels(images, labels, correct_labels, indices):
    fig, axes = plt.subplots(5, 20, figsize=(10, 3.5))
    axes = axes.ravel()

    for i in range(100):
        k = indices[i]
        axes[i].imshow(images[k].reshape(28, 28), cmap='gray')
        axes[i].set_title(f"{DIGITS[labels[k]]}", fontsize=9, color='black' if labels[k] == correct_labels[k] else 'red')
        axes[i].axis('off')

    plt.subplots_adjust(wspace=0.5)

indices = [_ for _ in range(len(test_x))]
np.random.shuffle(indices)
plot_images_with_labels(test_x, model.predict(test_x).argmax(axis=1), test_y, indices)

## Konvertierung des Modells

Dein trainiertes Modell besteht nun aus allen Gewichten und Bias-Werten – es befindet sich in der Datei `my_keras_model.h5`.

Wir möchten nun das neuronale Netz interaktiv testen, wofür sich JavaScript im Browser anbietet, da wir hier leicht Eingaben mit der Maus machen können. Dafür müssen wir das Modell zunächst in ein Format konvertieren, das von JavaScript gelesen werden kann. Gehe hierzu auf die Seite [https://tf.hackschule.de/](https://tf.hackschule.de/) und konvertiere dein Modell.

Du erhältst eine ZIP-Datei, in der sich zwei Dateien befinden. Diese musst du beide in das Verzeichnis `05-model` kopieren. Stelle anschließend sicher, dass sich die drei Dateien `index.html`, `index.js` und `worker.js` im Projektverzeichnis befinden (also nicht in `05-model`, sondern darüber – im selben Verzeichnis wie dieses Notebook).

## Interaktiver Test

Starte den HTTP-Server, indem du unten rechts auf »Live Server« klickst.

Du kannst nun im Browser links oben in der Fläche zeichnen (linke Maustaste: zeichnen, rechte Maustaste: löschen). Dabei kannst du beobachten, wir die Informationen durch das künstliche neuronale Netzwerk laufen und am Ende eine Vorhersage getroffen wird, um welche Ziffer es sich handelt.

_Eigentlich gibt es ein Kommanozeilen-Programm, das diese Konvertierung durchführt und das ist auch genau das, was auf tf.hackschule.de passiert. Ich habe das Tool in der Schule nur nicht zum Laufen bekommen, weshalb ich schnell diese kleine Seite gebastelt habe, die den Job übernimmt._ 😉